# Lab PySpark

In [0]:
#imports
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml import Pipeline

## Load data

In [0]:
df=spark.read.csv('/FileStore/tables/Heart.csv', header=True, inferSchema=True)
df.show(5)

+----------+---+---+------------+------+----+---+-------+-----+-----+-------+-----+---+----------+---+
Unnamed: 0|Age|Sex| ChestPain|RestBP|Chol|Fbs|RestECG|MaxHR|ExAng|Oldpeak|Slope| Ca| Thal|AHD|
+----------+---+---+------------+------+----+---+-------+-----+-----+-------+-----+---+----------+---+
 1| 63| 1| typical| 145| 233| 1| 2| 150| 0| 2.3| 3|0.0| fixed| No|
 2| 67| 1|asymptomatic| 160| 286| 0| 2| 108| 1| 1.5| 2|3.0| normal|Yes|
 3| 67| 1|asymptomatic| 120| 229| 0| 2| 129| 1| 2.6| 2|2.0|reversable|Yes|
 4| 37| 1| nonanginal| 130| 250| 0| 0| 187| 0| 3.5| 3|0.0| normal| No|
 5| 41| 0| nontypical| 130| 204| 0| 2| 172| 0| 1.4| 1|0.0| normal| No|
+----------+---+---+------------+------+----+---+-------+-----+-----+-------+-----+---+----------+---+
only showing top 5 rows

In [0]:
#change all column names from camel style to all lower case
for col in df.columns:
    df=df.withColumnRenamed(col, col.lower())
df.show(5)

+----------+---+---+------------+------+----+---+-------+-----+-----+-------+-----+---+----------+---+
unnamed: 0|age|sex| chestpain|restbp|chol|fbs|restecg|maxhr|exang|oldpeak|slope| ca| thal|ahd|
+----------+---+---+------------+------+----+---+-------+-----+-----+-------+-----+---+----------+---+
 1| 63| 1| typical| 145| 233| 1| 2| 150| 0| 2.3| 3|0.0| fixed| No|
 2| 67| 1|asymptomatic| 160| 286| 0| 2| 108| 1| 1.5| 2|3.0| normal|Yes|
 3| 67| 1|asymptomatic| 120| 229| 0| 2| 129| 1| 2.6| 2|2.0|reversable|Yes|
 4| 37| 1| nonanginal| 130| 250| 0| 0| 187| 0| 3.5| 3|0.0| normal| No|
 5| 41| 0| nontypical| 130| 204| 0| 2| 172| 0| 1.4| 1|0.0| normal| No|
+----------+---+---+------------+------+----+---+-------+-----+-----+-------+-----+---+----------+---+
only showing top 5 rows

### Convert categorical features `ChestPain` and `Thal` into numerical columns

In [0]:
num_features=[name for name, type in df.dtypes if (type in ['int','double']) & (name != 'unnamed: 0')]
num_df=df.select(num_features).show(5)

+---+---+------+----+---+-------+-----+-----+-------+-----+---+
age|sex|restbp|chol|fbs|restecg|maxhr|exang|oldpeak|slope| ca|
+---+---+------+----+---+-------+-----+-----+-------+-----+---+
 63| 1| 145| 233| 1| 2| 150| 0| 2.3| 3|0.0|
 67| 1| 160| 286| 0| 2| 108| 1| 1.5| 2|3.0|
 67| 1| 120| 229| 0| 2| 129| 1| 2.6| 2|2.0|
 37| 1| 130| 250| 0| 0| 187| 0| 3.5| 3|0.0|
 41| 0| 130| 204| 0| 2| 172| 0| 1.4| 1|0.0|
+---+---+------+----+---+-------+-----+-----+-------+-----+---+
only showing top 5 rows

In [0]:
num_features+['chestpain']

Out[15]: ['age',
 'sex',
 'restbp',
 'chol',
 'fbs',
 'restecg',
 'maxhr',
 'exang',
 'oldpeak',
 'slope',
 'ca',
 'chestpain']

--------------------------------------------------------------------------- 
 TypeError Traceback (most recent call last)
 /databricks/spark/python/pyspark/ml/param/__init__.py in _set (self, **kwargs) 
 463 try : 
 --> 464 value = p . typeConverter ( value ) 
 465 except TypeError as e : 

 /databricks/spark/python/pyspark/ml/param/__init__.py in toListString (value) 
 158 return [ TypeConverters . toString ( v ) for v in value ] 
 --> 159 raise TypeError ( "Could not convert %s to list of strings" % value ) 
 160 

 TypeError : Could not convert ['StringIndexer_a7406410b9fd__output', ['age', 'sex', 'restbp', 'chol', 'fbs', 'restecg', 'maxhr', 'exang', 'oldpeak', 'slope', 'ca']] to list of strings

During handling of the above exception, another exception occurred:

 TypeError Traceback (most recent call last)
 <command-38855650394219> in <module> 
 1 indexer = StringIndexer ( inputCols = [ 'chestpain' , 'thal' ] , outputCols = [ 'chestpainIndex' , 'thalIndex' ] ) 
 2 inputs = [ indexer . getOutputCol ( ) , num_features ] 
 ----> 3 encoder = OneHotEncoder ( inputCols = inputs , outputCols = [ 'chestpainVec1' , 'thalVec2' ] + num_features ) 
 4 pipeline = Pipeline ( stages = [ indexer , encoder ] ) 
 5 pipeline . fit ( df ) . transform ( df ) . show ( ) 

 /databricks/spark/python/pyspark/__init__.py in wrapper (self, *args, **kwargs) 
 112 raise TypeError ( "Method %s forces keyword arguments." % func . __name__ ) 
 113 self . _input_kwargs = kwargs
 --> 114 return func ( self , ** kwargs ) 
 115 return wrapper
 116 

 /databricks/spark/python/pyspark/ml/feature.py in __init__ (self, inputCols, outputCols, handleInvalid, dropLast, inputCol, outputCol) 
 2552 "org.apache.spark.ml.feature.OneHotEncoder", self.uid)
 2553 kwargs = self . _input_kwargs
 -> 2554 self . setParams ( ** kwargs ) 
 2555 
 2556 @ keyword_only

 /databricks/spark/python/pyspark/__init__.py in wrapper (self, *args, **kwargs) 
 112 raise TypeError ( "Method %s forces keyword arguments." % func . __name__ ) 
 113 self . _input_kwargs = kwargs
 --> 114 return func ( self , ** kwargs ) 
 115 return wrapper
 116 

 /databricks/spark/python/pyspark/ml/feature.py in setParams (self, inputCols, outputCols, handleInvalid, dropLast, inputCol, outputCol) 
 2564 """
 2565 kwargs = self . _input_kwargs
 -> 2566 return self . _set ( ** kwargs ) 
 2567 
 2568 @ since ( "2.3.0" ) 

 /databricks/spark/python/pyspark/ml/param/__init__.py in _set (self, **kwargs) 
 464 value = p . typeConverter ( value ) 
 465 except TypeError as e : 
 --> 466 raise TypeError ( 'Invalid param value given for param "%s". %s' % ( p . name , e ) ) 
 467 self . _paramMap [ p ] = value
 468 return self

 TypeError : Invalid param value given for param "inputCols". Could not convert ['StringIndexer_a7406410b9fd__output', ['age', 'sex', 'restbp', 'chol', 'fbs', 'restecg', 'maxhr', 'exang', 'oldpeak', 'slope', 'ca']] to list of strings